In [43]:
### Updating the performance.py for my study
import os
import cv2
import time
import numpy as np
import pandas as pd
import imgaug as ia
from utils.iou import iou
from imgaug import augmenters as iaa
from yolo.detector import YOLO
from fingertip import Fingertips

#hand = YOLO('weights/yolo.h5', threshold=0.5)
fingertip = Fingertips(model='vgg', weights='weights/weights025.h5')
resolution = 1


image_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/for_predict_check/' 
label_directory = 'C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/label/'
image_files = os.listdir(image_directory)

""" Ground truth label file for Parkinson's data """
labels_data = pd.read_csv(label_directory + "combined_labels_v3_cleaned.csv")
labels_data["filename"] = labels_data["video_name"] + "@" + labels_data.seq.str[3:6].astype(int).astype(str)


total_error = np.zeros([1, 4])
avg_hand_detect_time = 0
avg_fingertip_detect_time = 0
avg_time = 0
avg_iou = 0
count = 0
distance_error = []

# height = int(480 * resolution)
# width = int(640 * resolution)

for image_name in image_files:
    """ Getting the ground truth labels """
    image = cv2.imread(image_directory + image_name)
    orig_image = np.copy(image)
    cols, rows, _ = image.shape
    if count == 0:
        print("row dim", rows)
        print("col dim", cols)
    filename = image_name.split(".jpg")[0]
    
    """ Predictions for the test images """
    # Resizing the image to a square dimension - Kanav
    old_size = image.shape[:2]
    desired_size = max(old_size)
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    new_size
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0,0,0]
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    
    orig_padded_image = np.copy(image)
    
    gt = []

    #fetching the groundtruth labels and adjusting them for padding 
    xi = (labels_data[labels_data["filename"] == filename]["new_finger_x"]) + left
    yi = (labels_data[labels_data["filename"] == filename]["new_finger_y"]) + top 
    xt = (labels_data[labels_data["filename"] == filename]["new_thumb_x"]) + left
    yt = (labels_data[labels_data["filename"] == filename]["new_thumb_y"]) + top

    # standardising the gt labels
    xi = (xi/max(old_size))
    yi = (yi/max(old_size)) 
    xt = (xt/max(old_size)) 
    yt = (yt/max(old_size)) 
    

    
    gt = [xi, yi, xt, yt] #first index and then thumb


    #resizing image as per model requirements - change this when you try some other size - Kanav        
    image = cv2.resize(image, (128, 128), interpolation=cv2.INTER_LINEAR)

    tic3 = time.time()
    position = fingertip.classify(image=image)
    toc3 = time.time()
    avg_fingertip_detect_time = avg_fingertip_detect_time + (toc3 - tic3)
    
    

    """ Drawing bounding box and fingertip """
#     #plotting on 128 *128 padded image - red & green are predicted
#     image = cv2.circle(image, (int(position[0]*128), int(position[1]*128)), 2, (0, 0, 255), -1)
#     image = cv2.circle(image, (int(position[2]*128), int(position[3]*128)), 2, (0, 255, 0), -1)
    
#     image = cv2.circle(image, (int(gt[0]*128), int(gt[1]*128)), 2, (255, 255, 0), -1)
#     image = cv2.circle(image, (int(gt[2]*128), int(gt[3]*128)), 2, (0, 255, 255), -1)
#     #cv2.imshow('predicted_image', image)
#     cv2.imwrite('C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/predict_out/128_predicted/' 
#                 + filename + '_out.jpg', image)

#     #drawing on 910*910 padded image
#     orig_padded_image = cv2.circle(orig_padded_image, (int(position[0]*910), int(position[1]*910)), 10, (0, 0, 255), -1)
#     orig_padded_image = cv2.circle(orig_padded_image, (int(position[2]*910), int(position[3]*910)), 10, (0, 255, 0), -1)
    
#     orig_padded_image = cv2.circle(orig_padded_image, (int(gt[0]*910), int(gt[1]*910)), 10, (255, 255, 0), -1)
#     orig_padded_image = cv2.circle(orig_padded_image, (int(gt[2]*910), int(gt[3]*910)), 10, (0, 255, 255), -1)

#     #cv2.imshow('predicted_image', image)
#     cv2.imwrite('C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/predict_out/orig_padded_predicted/' 
#                  + filename + '_out.jpg', orig_padded_image)

    #drawing on 780*910 non-padded image
    orig_image = cv2.circle(orig_image, (int(position[0]*910-65), int(position[1]*910)), 10, (0, 0, 255), -1)
    orig_image = cv2.circle(orig_image, (int(position[2]*910-65), int(position[3]*910)), 10, (0, 255, 0), -1)
    
    orig_image = cv2.circle(orig_image, (int(gt[0]*910-65), int(gt[1]*910)), 10, (255, 255, 0), -1)
    orig_image = cv2.circle(orig_image, (int(gt[2]*910-65), int(gt[3]*910)), 10, (0, 255, 255), -1)

    cv2.imwrite('C:/Users/Kanav/Documents/Dissertation/Parkinsons_Disease/Codes/Fingertip-Mixed-Reality/Dataset/predict_out/orig/' 
                + filename + '_out.jpg', orig_image)

    cv2.waitKey(0)
    
    

row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col dim 910
row dim 780
col 

In [21]:
image

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [18]:
position*910

array([343.1733 , 495.86758, 333.2247 , 526.1512 ], dtype=float32)

In [2]:
import numpy
import cv2
#from yolo.detector import YOLO
#from fingertip import Fingertips

# hand = YOLO(weights='weights/yolo.h5', threshold=0.5)
# fingertip = Fingertips(model='vgg', weights='weights/vgg16.h5')

image_file = '..\test_kanav_del\img036.png'
image = cv2.imread(image_file)

cv2.imshow('uploaded', image)
#tl, br = hand.detect(image=image)

# if tl and br is not None:
#     alpha_x = 0
#     alpha_y = 0
#     tl = (tl[0] - alpha_x, tl[1] - alpha_y)
#     br = (br[0] + alpha_x, br[1] + alpha_y)

#     xmin = int(tl[0])
#     ymin = int(tl[1])
#     xmax = int(br[0])
#     ymax = int(br[1])

#     ymin = ymin if ymin > 0 else 0
#     xmin = xmin if xmin > 0 else 0

#     cropped_image = image[ymin:ymax, xmin:xmax]
#     cols, rows, _ = cropped_image.shape
#     cropped_image = cv2.resize(cropped_image, (128, 128))
#     # cv2.imwrite('cropped_image.jpg', cropped_image)
#     position = fingertip.classify(image=cropped_image)

#     position[0] = (position[0]) * rows + tl[0]
#     position[1] = (position[1]) * cols + tl[1]
#     position[2] = (position[2]) * rows + tl[0]
#     position[3] = (position[3]) * cols + tl[1]

#     """ Drawing bounding box and fingertip """
#     image = cv2.rectangle(image, tl, br, (235, 26, 158), 4, 1)
#     image = cv2.circle(image, (int(position[0]), int(position[1])), 14, (0, 0, 255), -1)
#     image = cv2.circle(image, (int(position[2]), int(position[3])), 14, (0, 255, 0), -1)
#     cv2.imshow('predicted_image', image)
#     cv2.imwrite('data/' + image_file[:-4] + '_out.jpg', image)
#     cv2.waitKey(0)


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\Kanav\anaconda3\envs\myenv\python.exe"
  * The NumPy version is: "1.21.6"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.
